In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential

In [2]:
data = pd.read_csv('kddcup.data - Copy.csv',header=None)

In [3]:
data.shape

(4898431, 42)

In [4]:
data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41],
           dtype='int64')

In [5]:
from collections import Counter

In [6]:
Counter(data.iloc[:,-1])

Counter({'normal.': 972781,
         'buffer_overflow.': 30,
         'loadmodule.': 9,
         'perl.': 3,
         'neptune.': 1072017,
         'smurf.': 2807886,
         'guess_passwd.': 53,
         'pod.': 264,
         'teardrop.': 979,
         'portsweep.': 10413,
         'ipsweep.': 12481,
         'land.': 21,
         'ftp_write.': 8,
         'back.': 2203,
         'imap.': 12,
         'satan.': 15892,
         'phf.': 4,
         'nmap.': 2316,
         'multihop.': 7,
         'warezmaster.': 20,
         'warezclient.': 1020,
         'spy.': 2,
         'rootkit.': 10})

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 42 columns):
0     int64
1     object
2     object
3     object
4     int64
5     int64
6     int64
7     int64
8     int64
9     int64
10    int64
11    int64
12    int64
13    int64
14    int64
15    int64
16    int64
17    int64
18    int64
19    int64
20    int64
21    int64
22    int64
23    int64
24    float64
25    float64
26    float64
27    float64
28    float64
29    float64
30    float64
31    int64
32    int64
33    float64
34    float64
35    float64
36    float64
37    float64
38    float64
39    float64
40    float64
41    object
dtypes: float64(15), int64(23), object(4)
memory usage: 1.5+ GB


In [13]:
sum(data.isnull())

861

In [14]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [16]:
data.describe()

,0,4,5,6,7,8,9,10,11,12,...,31,32,33,34,35,36,37,38,39,40
count,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,...,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06
mean,4.834243e+01,1.834621e+03,1.093623e+03,5.716116e-06,6.487792e-04,7.961733e-06,1.243766e-02,3.205108e-05,1.435290e-01,8.088304e-03,...,2.329811e+02,1.892142e+02,7.537132e-01,3.071111e-02,6.050520e-01,6.464107e-03,1.780911e-01,1.778859e-01,5.792780e-02,5.765941e-02
std,7.233298e+02,9.414311e+05,6.450123e+05,2.390833e-03,4.285434e-02,7.215084e-03,4.689782e-01,7.299408e-03,3.506116e-01,3.856481e+00,...,6.402094e+01,1.059128e+02,4.111860e-01,1.085432e-01,4.809877e-01,4.125978e-02,3.818382e-01,3.821774e-01,2.309428e-01,2.309777e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,4.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,4.900000e+01,4.100000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,5.200000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,2.550000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,1.032000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,2.550000e+02,1.000000e+00,4.000000e-02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.832900e+04,1.379964e+09,1.309937e+09,1.000000e+00,3.000000e+00,1.400000e+01,7.700000e+01,5.000000e+00,1.000000e+00,7.479000e+03,...,2.550000e+02,2.550000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [20]:
data_num = pd.DataFrame({col: data[col].astype('category').cat.codes for col in data}, index=data.index)

In [21]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 42 columns):
0     int16
1     int8
2     int8
3     int8
4     int16
5     int16
6     int8
7     int8
8     int8
9     int8
10    int8
11    int8
12    int8
13    int8
14    int8
15    int8
16    int8
17    int8
18    int8
19    int8
20    int8
21    int8
22    int16
23    int16
24    int8
25    int8
26    int8
27    int8
28    int8
29    int8
30    int8
31    int16
32    int16
33    int8
34    int8
35    int8
36    int8
37    int8
38    int8
39    int8
40    int8
41    int8
dtypes: int16(7), int8(35)
memory usage: 228.9 MB


In [24]:
feature = data_num.iloc[:,:-1]

In [25]:
label = data_num.iloc[:,-1]

In [27]:
Counter(label)

Counter({11: 972781,
         1: 30,
         7: 9,
         12: 3,
         9: 1072017,
         18: 2807886,
         3: 53,
         14: 264,
         20: 979,
         15: 10413,
         5: 12481,
         6: 21,
         2: 8,
         0: 2203,
         4: 12,
         17: 15892,
         13: 4,
         10: 2316,
         8: 7,
         22: 20,
         21: 1020,
         19: 2,
         16: 10})

In [31]:
scaler = StandardScaler()
feautre_scaler = scaler.fit_transform(feature)

C:\Users\hassan\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int8, int16 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\hassan\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int8, int16 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(feature,label,
                                                    test_size=0.25,
                                                    random_state=101)

In [35]:
X_train.shape, X_test.shape

((3673823, 41), (1224608, 41))

In [36]:
y_train.shape, y_test.shape

((3673823,), (1224608,))

In [41]:
X_train_arr = np.array(X_train)
y_train_arr = np.array(y_train)
#test
X_test_arr = np.array(X_test)
y_test_arr = np.array(y_test)

In [43]:
train_reshape = np.reshape(X_train_arr, (X_train_arr.shape[0],X_train_arr.shape[1],1))
y_train_arr_cat = to_categorical(y_train_arr)

In [44]:
test_reshape = np.reshape(X_test_arr, (X_test_arr.shape[0],X_test_arr.shape[1],1))
y_test_arr_cat = to_categorical(y_test_arr)

In [48]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(41, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(23, activation="softmax"))
print(cnn.summary())

C:\Users\hassan\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(41, 1), padding="same")`
  
C:\Users\hassan\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 41, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 23)                2967      
Total params: 167,191
Trainable params: 167,191
Non-trainable params: 0
_________________________________________________________________
None

In [49]:
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [ ]:
cnn.fit(train_reshape, y_train_arr_cat, epochs=1)

Epoch 1/1
 440480/3673823 [==>...........................] - ETA: 13:47 - loss: 0.3458 - acc: 0.9783